This notebook implements a test flow described in the following [use case](https://github.com/interfacerproject/Interfacer-notebook/blob/main/gownshirt_flow.1.1.txt) .

In [ ]:
# The following should reload if_lib if we change it on disk
%load_ext autoreload
%aimport if_lib, if_utils, if_dpp, if_graphics, if_consts
%autoreload 1
import os
import json
import random

from if_utils import get_filename, show_data, save_traces

from if_lib import generate_random_challenge, read_HMAC, read_keypair, get_id_person, get_location_id, \
get_unit_id, get_resource_spec_id, get_resource, get_process, create_event, make_transfer, reduce_resource, set_user_location

from if_dpp import trace_query, check_traces, er_before, get_dpp

from if_graphics import vis_dpp, make_sankey, consol_trace

In [ ]:
# We define the constant for our case
USE_CASE = 'gownshirt'

# What endpoint are we talking to?
# debug
# ENDPOINT = 'http://65.109.11.42:10000/api'
ENDPOINT = 'http://zenflows-debug.interfacer.dyne.org/api'
# staging
# ENDPOINT = 'http://65.109.11.42:8000/api'
# ENDPOINT = 'https://zenflows-staging.interfacer.dyne.org/api'
# testing
# ENDPOINT = 'http://65.109.11.42:9000/api'
# ENDPOINT = 'https://zenflows-test.interfacer.dyne.org/api'
USERS = ['alice', 'carol', 'bob']

In [ ]:
# Calculate names of settings files
USERS_FILE = get_filename('cred_users.json', ENDPOINT, USE_CASE)
LOCS_FILE = get_filename('loc_users.json', ENDPOINT, USE_CASE)
UNITS_FILE = get_filename('units_data.json', ENDPOINT, USE_CASE)
SPECS_FILE = get_filename('res_spec_data.json', ENDPOINT, USE_CASE)

In [ ]:
# Read or define user data that is going to be used in the GraphQL calls

# create data structure to hold processes
process_data = {}

# create data structures to hold resources and events (possibly to compare results from track and trace)
res_data = {}
event_seq = []

if os.path.isfile(USERS_FILE):
    with open(USERS_FILE,'r') as f:
        users_data = json.loads(f.read())
    print("Credentials file available for users")
else:
    users_data = {}
    users_data['alice'] = {
      "userChallenges": {
        "whereParentsMet": "London",
        "nameFirstPet": "Fuffy",
        "nameFirstTeacher": "Jim",
        "whereHomeTown": "Paris",
        "nameMotherMaid": "Wright"
      },
      "name": "Alice",
      "username": "alice_username",
      "email": "alice@example.org",
      "note": "me.alice.org"
    }
    users_data['carol'] = {
        "userChallenges": {
            "whereParentsMet":"Amsterdam",
            "nameFirstPet":"Toby",
            "nameFirstTeacher":"Juliet",
            "whereHomeTown":"Rome",
            "nameMotherMaid":"Banks"
        },
        "name": "Carol",
        "username": "carol_username",
        "email": "carol@example.org",
        "note" : "me.carol.org"
    }

    users_data['bob'] = {
        "userChallenges": {
            "whereParentsMet":"Rome",
            "nameFirstPet":"Ku",
            "nameFirstTeacher":"George",
            "whereHomeTown":"Florence",
            "nameMotherMaid":"Canti"
        },
        "name": "Bob",
        "username": "bob_username",
        "email": "bob@example.org",
        "note" : "me.bob.org"
    }

    with open(USERS_FILE,'w') as f:
        json.dump(users_data, f)

if os.path.isfile(LOCS_FILE):
    with open(LOCS_FILE,'r') as f:
        locs_data = json.loads(f.read())
    print("Location file available")
else:
    locs_data = {}
    locs_data['alice'] = {
        "name": "Dyne",
        "lat": 52.39679,
        "long": 4.8781073,
        "addr": "Haparandadam 7, A1, 1013 AK Amsterdam",
        "note": "location.alice.org"
    }
    locs_data['carol'] = {
        "name": "CleanLease",
        "lat" : 51.47240440868687,
        "long" : 5.412460440524406,
        "addr" : "De schakel 30, 5651 Eindhoven",
        "note": "location.carol.org"
    }
    locs_data['bob'] = {
        "name": "Fab Lab Hamburg",
        "lat" : 51.1531305,
        "long" : 5.2685045,
        "addr" : "Stockmeyerstr. 43 Halle 4K Eingang von der Wasserseite, 20457 Hamburg, Germany",
        "note": "location.bob.org"
    }

    with open(LOCS_FILE,'w') as f:
        json.dump(locs_data, f)

if os.path.isfile(UNITS_FILE):
    with open(UNITS_FILE,'r') as f:
        units_data = json.loads(f.read())
    print(f"Unit file available")
else:
    units_data = {}
#     with open(file,'w') as f:
#         json.dump(units_data, f)


if os.path.isfile(SPECS_FILE):
    with open(SPECS_FILE,'r') as f:
        res_spec_data = json.loads(f.read())
    print(f"Resource Spec file available")
else:
    res_spec_data = {}



In [ ]:
# Read HMAC or get it from the server
for user in USERS:
    read_HMAC(USERS_FILE, users_data, user, endpoint=ENDPOINT)


In [ ]:
# Read the keypair
for user in USERS:
    read_keypair(USERS_FILE, users_data, user)

In [ ]:
# read or get id of the person
for user in USERS:
    get_id_person(USERS_FILE, users_data, user, endpoint=ENDPOINT)

In [ ]:
# Read of get the location id
for user in USERS:
    get_location_id(LOCS_FILE, users_data[user], locs_data, user, endpoint=ENDPOINT)
    set_user_location(USERS_FILE, users_data, locs_data, user, endpoint=ENDPOINT)

In [ ]:
# Get the ids of all units
get_unit_id(UNITS_FILE, users_data['alice'], units_data, 'piece', 'u_piece', 'om2:one', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['alice'], units_data, 'mass', 'kg', 'om2:kilogram', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['alice'], units_data, 'volume', 'lt', 'om2:litre', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['alice'], units_data, 'time', 'h', 'om2:hour', endpoint=ENDPOINT)
get_unit_id(UNITS_FILE, users_data['alice'], units_data, 'distance', 'km', 'om2:kilometer', endpoint=ENDPOINT)


In [ ]:
# Create the processes

# Create the process that wraps cleaning the gown
process_name = 'Clean_gown_1'
user_data = users_data['alice']
note = f"Clean gown first time process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

process_name = 'Clean_gown_2'
user_data = users_data['alice']
note = f"Clean gown second time process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps cleaning the gown and shirt
process_name = 'Clean_textile'
user_data = users_data['alice']
note = f"Clean gown and shirt process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps sewing of the gown (its creation)
process_name = 'Create_gown'
user_data = users_data['alice']
note = f"Create gown process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps using the gown in the hospital and making it dirty
process_name = 'Do_surgery'
user_data = users_data['alice']
note = f"Use gown in surgery process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps creating the shirt design
process_name = 'Create_shirt_design'
user_data = users_data['carol']
note = f"Create shirt design process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps creating the shirt design
process_name = 'Create_shirt'
user_data = users_data['carol']
note = f"Create shirt process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

# Create the process that wraps making the cotton insulation
process_name = 'Make_cotton_insulation'
user_data = users_data['alice']
note = f"Create make cotton insulation process performed by {user_data['name']}"
get_process(process_name, process_data, note, user_data, endpoint=ENDPOINT)

In [ ]:
# Read all the resource specifications
name = 'gown'
note = 'Specification for gowns'
classification = 'https://www.wikidata.org/wiki/Q89990310'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'soap'
note = 'Specification for soap to be used to wash the gowns'
classification = 'https://www.wikidata.org/wiki/Q34396'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'cotton'
note = 'Specification for cotton'
classification = 'https://www.wikidata.org/wiki/Q11457'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'water'
note = 'Specification for water'
classification = 'https://www.wikidata.org/wiki/Q283'
default_unit_id = units_data['volume']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'surgery_work'
note = 'Specification for surgical work'
classification = 'https://www.wikidata.org/wiki/Q600236'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'transport_service'
note = 'Specification for transport service'
classification = 'https://www.wikidata.org/wiki/Q600236'
default_unit_id = units_data['distance']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'sewing_machine'
note = 'Specification for sewing machine'
classification = 'https://www.wikidata.org/wiki/Q106153893'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'sewing_machine_usage'
note = 'Specification for sewing machine usage'
classification = 'https://www.wikidata.org/wiki/Q106153893'
default_unit_id = units_data['time']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'shirt'
note = 'Specification for shirts'
classification = 'https://www.wikidata.org/wiki/Q76768'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'shirt_design'
note = 'Specification for shirt designs'
classification = 'https://www.me.org/designs/shirt_12'
default_unit_id = units_data['piece']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'cotton_scraps'
note = 'Specification for cotton scraps'
classification = 'https://www.wikidata.org/wiki/Q72134860'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'cotton_waste'
note = 'Specification for cotton waste'
classification = 'https://www.wikidata.org/wiki/Q39915841'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)

name = 'cotton_insulation'
note = 'Specification for cotton insulation'
classification = 'https://www.wikidata.org/wiki/Q918306'
default_unit_id = units_data['mass']['id']
get_resource_spec_id(SPECS_FILE, users_data['alice'], res_spec_data, name, note, classification, default_unit_id, endpoint=ENDPOINT)


In [ ]:
# We create the resources that will not be saved to file as it is assumed they are recreated at each run

# gown will be produced

res_name = 'cotton'
amount = 100
get_resource(res_data, res_spec_data, res_name, users_data['alice'], event_seq, amount, endpoint=ENDPOINT)

res_name = 'water'
amount = 60
get_resource(res_data, res_spec_data, res_name, users_data['alice'], event_seq, amount, endpoint=ENDPOINT)

res_name = 'soap'
amount = 30
get_resource(res_data, res_spec_data, res_name, users_data['alice'], event_seq, amount, endpoint=ENDPOINT)

res_name = 'sewing_machine'
amount = 1
get_resource(res_data, res_spec_data, res_name, users_data['carol'], event_seq, amount, endpoint=ENDPOINT)

# res_name = 'surgery_work'
# amount = 100
# get_resource(res_data, res_spec_data, res_name, users_data['alice'], event_seq, amount)



In [ ]:
# Define event consume for the gown creation
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume cotton for gown'
amount = 25
cur_pros = process_data['Create_gown']
cur_res = res_data['cotton_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})

In [ ]:
# Define event produce for the gown creation
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce gown'
amount = 1
cur_pros = process_data['Create_gown']

res_data['gown_res'] = {
    "res_ref_id": f'gown-{random.randint(0, 10000)}',
    "name": 'gown',
    "spec_id": res_spec_data['gown']['id']
}
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Define event produce for cotton scraps byproduct
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce cotton scraps'
amount = 7
cur_pros = process_data['Create_gown']

res_data['cotton_scraps_res'] = {
    "res_ref_id": f'cotton_scraps-{random.randint(0, 10000)}',
    "name": 'cotton_scraps',
    "spec_id": res_spec_data['cotton_scraps']['id']
}
cur_res = res_data['cotton_scraps_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# accept the gown for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'accept'
event_note='accept gowns to be cleaned'
amount = 1
cur_pros = process_data['Clean_gown_1']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume water for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume water for the washing'
amount = 20
cur_pros = process_data['Clean_gown_1']
cur_res = res_data['water_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# consume soap for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume soap for the washing'
amount = 10
cur_pros = process_data['Clean_gown_1']
cur_res = res_data['soap_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# modify the gown that is now clean
cur_res = action = event_note = amount = cur_pros = None
action = 'modify'
event_note='modify clean after washing'
amount = 1
cur_pros = process_data['Clean_gown_1']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})



In [ ]:
# Transfer the gown from the owner/leaser to the hospital
cur_res = action = event_note = amount = cur_pros = None
note='Transfer gowns from Alice to Bob'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['alice'], action, note, users_data['bob'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Use the gown to perform surgery and as a consequence make it dirty
cur_res = action = event_note = amount = cur_pros = None
action = 'accept'
event_note='accept use for surgery'
amount = 1
cur_pros = process_data['Do_surgery']
cur_res = res_data['gown_res']

event_id, ts = create_event(users_data['bob'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
   


In [ ]:
# Work with the gown to perform surgery
cur_res = action = event_note = amount = cur_pros = None
action = 'work'
event_note='work perform surgery'

cur_pros = process_data['Do_surgery']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['surgery_work']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['surgery_work']['id']
effort_spec['amount'] = 5

event_id, ts = create_event(users_data['bob'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Modify the gown and make it dirty as a consequence of being used
cur_res = action = event_note = amount = cur_pros = None
action = 'modify'
event_note='modify dirty after surgery'
amount = 1
cur_pros = process_data['Do_surgery']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['bob'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the gown from Bob to Alice
cur_res = action = event_note = amount = cur_pros = None
note='Transfer gowns to Alice'
action = 'transfer-custody'
amount = 1
cur_res = res_data['gown_res']

event_id, ts = make_transfer(users_data['bob'], action, note, users_data['alice'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# accept the gown for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'accept'
event_note='accept gowns to be cleaned'
amount = 1
cur_pros = process_data['Clean_gown_2']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume water for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume water for the washing'
amount = 20
cur_pros = process_data['Clean_gown_2']
cur_res = res_data['water_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# consume soap for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume soap for the washing'
amount = 10
cur_pros = process_data['Clean_gown_2']
cur_res = res_data['soap_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# modify the gown that is now clean
cur_res = action = event_note = amount = cur_pros = None
action = 'modify'
event_note='modify clean after washing'
amount = 1
cur_pros = process_data['Clean_gown_2']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer 10 kg of cotton from from Alice to Carol
cur_res = action = event_note = amount = cur_pros = None
note='Transfer cotton to Carol'
action = 'transfer'
amount = 10
cur_res = res_data['cotton_res']

event_id, ts = make_transfer(users_data['alice'], action, note, users_data['carol'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Work with the gown to perform surgery
cur_res = action = event_note = amount = cur_pros = None
action = 'work'
event_note='work at shirt design'

cur_pros = process_data['Create_shirt_design']
effort_spec = {}
effort_spec['unit_id'] = res_spec_data['shirt_design']['defaultUnit']
effort_spec['spec_id'] = res_spec_data['shirt_design']['id']
effort_spec['amount'] = 5

event_id, ts = create_event(users_data['carol'], action, event_note, amount=0, process=cur_pros, \
                 res_spec_data=res_spec_data, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'amount': effort_spec['amount']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Define event produce for cotton scraps byproduct
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce shirt design'
amount = 1
cur_pros = process_data['Create_shirt_design']

res_data['shirt_design_res'] = {
    "res_ref_id": f'shirt_design-{random.randint(0, 10000)}',
    "name": 'shirt_design',
    "spec_id": res_spec_data['shirt_design']['id']
}
cur_res = res_data['shirt_design_res']


event_id, ts = create_event(users_data['carol'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume cotton for shirt
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume cotton for shirt'
amount = 5
cur_pros = process_data['Create_shirt']
cur_res = res_data['cotton_res']


event_id, ts = create_event(users_data['carol'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume cotton for shirt
cur_res = action = event_note = amount = cur_pros = None
action = 'cite'
event_note='cite shirt design'
amount = 1

cur_pros = process_data['Create_shirt']
cur_res = res_data['shirt_design_res']


event_id, ts = create_event(users_data['carol'], action, event_note, amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Use sewing machine for shirt
cur_res = action = event_note = amount = cur_pros = None
action = 'use'
event_note='use sewing machine for shirt'

cur_pros = process_data['Create_shirt']
cur_res = res_data['sewing_machine_res']
amount = 1

effort_spec = {}
effort_spec['unit_id'] = res_spec_data['sewing_machine_usage']['defaultUnit']
# effort_spec['spec_id'] = res_spec_data['sewing_machine_usage']['id']
effort_spec['amount'] = 3


event_id, ts = create_event(users_data['carol'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, effort_spec=effort_spec, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# Define event produce for cotton scraps byproduct
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce shirts'
amount = 2
cur_pros = process_data['Create_shirt']

res_data['shirt_res'] = {
    "res_ref_id": f'shirt-{random.randint(0, 10000)}',
    "name": 'shirt',
    "spec_id": res_spec_data['shirt']['id']
}
cur_res = res_data['shirt_res']


event_id, ts = create_event(users_data['carol'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# lower cotton (2 kg, Carol)
cur_res = action = event_note = amount = cur_pros = None
amount = 2
cur_res = res_data['cotton_res']

event_id, ts = reduce_resource(users_data['carol'], res_data=cur_res, res_spec_data=res_spec_data, amount=amount, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the shirt from Carol to Alice
cur_res = action = event_note = amount = cur_pros = None
note='Transfer shirt from Carol to Alice'
action = 'transfer'
amount = 1
cur_res = res_data['shirt_res']

event_id, ts = make_transfer(users_data['carol'], action, note, users_data['alice'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# accept the gown for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'accept'
event_note='accept gown to be cleaned'
amount = 1
cur_pros = process_data['Clean_textile']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# accept shirt for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'accept'
event_note='accept shirt to be cleaned'
amount = 1
cur_pros = process_data['Clean_textile']
cur_res = res_data['shirt_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})



In [ ]:
# consume water for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume water for the washing'
amount = 20
cur_pros = process_data['Clean_textile']
cur_res = res_data['water_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})

In [ ]:
# consume soap for washing
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume soap for the washing'
amount = 10
cur_pros = process_data['Clean_textile']
cur_res = res_data['soap_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})
event_seq.append({'ts': ts, 'process_id':cur_pros['id'], 'name' : cur_pros['name']})


In [ ]:
# modify the gown that is now clean
cur_res = action = event_note = amount = cur_pros = None
action = 'modify'
event_note='modify gown after washing'
amount = 1
cur_pros = process_data['Clean_textile']
cur_res = res_data['gown_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# modify the gown that is now clean
cur_res = action = event_note = amount = cur_pros = None
action = 'modify'
event_note='modify shirt after washing'
amount = 1
cur_pros = process_data['Clean_textile']
cur_res = res_data['shirt_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# consume cotton scraps for insulation
cur_res = action = event_note = amount = cur_pros = None
action = 'consume'
event_note='consume cotton scraps for insulation'
amount = 7
cur_pros = process_data['Make_cotton_insulation']
cur_res = res_data['cotton_scraps_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, existing_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Define event produce for cotton insulation
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce cotton insulation'
amount = 6
cur_pros = process_data['Make_cotton_insulation']

res_data['cotton_insulation_res'] = {
    "res_ref_id": f'cotton_insulation-{random.randint(0, 10000)}',
    "name": 'cotton_insulation',
    "spec_id": res_spec_data['cotton_insulation']['id']
}
cur_res = res_data['cotton_insulation_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Define event produce for cotton waste
cur_res = action = event_note = amount = cur_pros = None
action = 'produce'
event_note='produce cotton waste'
amount = 1
cur_pros = process_data['Make_cotton_insulation']

res_data['cotton_waste_res'] = {
    "res_ref_id": f'cotton_waste-{random.randint(0, 10000)}',
    "name": 'cotton_waste',
    "spec_id": res_spec_data['cotton_waste']['id']
}
cur_res = res_data['cotton_waste_res']


event_id, ts = create_event(users_data['alice'], action, event_note, amount=amount, process=cur_pros, \
                 res_spec_data=res_spec_data, new_res=cur_res, endpoint=ENDPOINT)

event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
# Transfer the shirt from Carol to Alice
cur_res = action = event_note = amount = cur_pros = None
note='Transfer cotton waste from Alice to Bob'
action = 'transfer'
amount = 1
cur_res = res_data['cotton_waste_res']

event_id, ts = make_transfer(users_data['alice'], action, note, users_data['bob'], amount, cur_res, locs_data, res_spec_data, endpoint=ENDPOINT)
event_seq.append({'ts': ts, 'event_id':event_id, 'action' : action, 'res_name': cur_res['name'], 'res': cur_res['id']})


In [ ]:
show_data(users_data, locs_data, res_data, units_data, res_spec_data, process_data, event_seq)

In [ ]:
trace_me = res_data['gown_res']['id']
print(f"Resource to be traced: {trace_me}")
tot_dpp = []
visited = set()
er_before(trace_me, users_data['alice'], dpp_children=tot_dpp, depth=0, visited=visited, endpoint=ENDPOINT)

# Serializing json
json_object = json.dumps(tot_dpp, indent=2)

print(json_object)
print(visited)

In [ ]:
be_dpp = get_dpp(trace_me, users_data['alice'], endpoint=ENDPOINT)
print(json.dumps(be_dpp, indent=2))

In [ ]:
trace = trace_query(trace_me, users_data['alice'], endpoint=ENDPOINT)
check_traces(trace, event_seq, tot_dpp, be_dpp)

In [ ]:
save_traces(USE_CASE, tot_dpp, trace, be_dpp, event_seq)

In [ ]:
labels = []
sources = []
targets = []
values = []
color_nodes = []
color_links = []
assigned = {}
vis_dpp(tot_dpp[0], count=0, assigned=assigned, labels=labels, targets=targets, sources=sources, values=values, color_nodes=color_nodes, color_links=color_links)
sources, targets = consol_trace(assigned, sources, targets)
make_sankey(sources, targets, labels, values, color_nodes, color_links)